In [120]:
from pprint import pprint
from pathlib import Path
import requests
import json
import pandas as pd
import csv
from dotenv import load_dotenv
import os
import hvplot.pandas
import calendar
import datetime
import bokeh

In [121]:
# read in the stocks (inventory) data .csv
inventorycsv = Path('../data/clean_data/corn_stocks_seasonal.csv')

inventory_df = pd.read_csv(inventorycsv, delimiter=',', parse_dates=True, infer_datetime_format=True)

In [122]:
# check out the header columns
inventory_df.tail()

,date,stock_report
78,2001-12-01,1040485000
79,2000-03-01,689660000
80,2000-06-01,437336000
81,2000-09-01,230059000
82,2000-12-01,966809000


In [123]:
# check for nulls and decide whether to drop any drop empty columns
inventory_df.isnull().sum()

# no null data

date            0
stock_report    0
dtype: int64

In [124]:
# Plot inventory levels over the past year
base_inv_plot = inventory_df.hvplot(kind='bar', title='Corn Inventory Volumes over Time', x='date', xlabel='Date', xticks=25, y='stock_report', ylabel='Inventory, Bu', yformatter='%.0f', rot=45)
base_inv_plot

:Bars   [date]   (stock_report)

In [125]:
# build inventory data exponential moving averages (1 year and 5 year)
window_1year = 4
window_5year = 20

ewma1_plot = inventory_df.ewm(span=window_1year).mean().hvplot(color='lightgreen', title='Corn Inventory Volumes over Time', x='date', xlabel='Date', xticks=25, y='stock_report', ylabel='Inventory, Bu', yformatter='%.0f', rot=45, legend=True)
ewma5_plot = inventory_df.ewm(span=window_5year).mean().hvplot(title='Corn Inventory Volumes over Time', x='date', xlabel='Date', xticks=25, y='stock_report', ylabel='Inventory, Bu', yformatter='%.0f', rot=45, legend=True)

base_inv_plot * ewma1_plot * ewma5_plot

:Overlay
   .Bars.I   :Bars   [date]   (stock_report)
   .Curve.I  :Curve   [date]   (stock_report)
   .Curve.II :Curve   [date]   (stock_report)

Not a great measure...will be statistically more relevant to look at the different quarters and compare to the previous quarters.

In [126]:
# build 4 dataframes to compare the 4 different quarters ()
months=[]
years=[]

for dates in inventory_df['date']:
    year = datetime.datetime.strptime(dates, '%Y-%m-%d').year
    month = calendar.month_name[datetime.datetime.strptime(dates, '%Y-%m-%d').month]
    months.append(month)
    years.append(year)

inventory_df['month'] = months
inventory_df['year'] = years

sept_df = inventory_df[inventory_df['month'] == 'September'].sort_values('date', ascending=True)
mar_df = inventory_df[inventory_df['month'] == 'March'].sort_values('date', ascending=True)
jun_df = inventory_df[inventory_df['month'] == 'June'].sort_values('date', ascending=True)
dec_df = inventory_df[inventory_df['month'] == 'December'].sort_values('date', ascending=True)

sept_plot = sept_df.hvplot(title='September Inventory Report Through Time', kind='bar', x='date', xlabel='Date', y='stock_report', ylabel='Inventory, Bu', xformatter=bokeh.models.formatters.DatetimeTickFormatter(), yformatter='%.0f', rot=45)

mar_plot = mar_df.hvplot(title='March Inventory Report Through Time', kind='bar', x='date', xlabel='Date', y='stock_report', ylabel='Inventory, Bu', xformatter=bokeh.models.formatters.DatetimeTickFormatter(), yformatter='%.0f', rot=45)

jun_plot = jun_df.hvplot(title='June Inventory Report Through Time', kind='bar', x='date', xlabel='Date', y='stock_report', ylabel='Inventory, Bu', xformatter=bokeh.models.formatters.DatetimeTickFormatter(), yformatter='%.0f', rot=45)

dec_plot = dec_df.hvplot(title='December Inventory Report Through Time', kind='bar', x='date', xlabel='Date', y='stock_report', ylabel='Inventory, Bu', xformatter=bokeh.models.formatters.DatetimeTickFormatter(), yformatter='%.0f', rot=45)

In [127]:
# calculate the moving averages for each season
# March
window_1year = 1
window_5year = 5

mar_ewma1_plot = mar_df.ewm(span=window_1year).mean().hvplot(color='red', x='date', xlabel='Date', y='stock_report', ylabel='Inventory, Bu', yformatter='%.0f', rot=45, legend=True)
mar_ewma5_plot = mar_df.ewm(span=window_5year).mean().hvplot(color='lightgreen', x='date', xlabel='Date', y='stock_report', ylabel='Inventory, Bu', yformatter='%.0f', rot=45, legend=True)

# June
jun_ewma1_plot = jun_df.ewm(span=window_1year).mean().hvplot(color='red', x='date', xlabel='Date', y='stock_report', ylabel='Inventory, Bu', yformatter='%.0f', rot=45, legend=True)
jun_ewma5_plot = jun_df.ewm(span=window_5year).mean().hvplot(color='lightgreen', x='date', xlabel='Date', y='stock_report', ylabel='Inventory, Bu', yformatter='%.0f', rot=45, legend=True)

# September
sept_ewma1_plot = sept_df.ewm(span=window_1year).mean().hvplot(color='red', x='date', xlabel='Date', y='stock_report', ylabel='Inventory, Bu', yformatter='%.0f', rot=45, legend=True)
sept_ewma5_plot = sept_df.ewm(span=window_5year).mean().hvplot(color='lightgreen', x='date', xlabel='Date', y='stock_report', ylabel='Inventory, Bu', yformatter='%.0f', rot=45, legend=True)

# December
dec_ewma1_plot = dec_df.ewm(span=window_1year).mean().hvplot(color='red', x='date', xlabel='Date', y='stock_report', ylabel='Inventory, Bu', yformatter='%.0f', rot=45, legend=True)
dec_ewma5_plot = dec_df.ewm(span=window_5year).mean().hvplot(color='lightgreen', x='date', xlabel='Date', y='stock_report', ylabel='Inventory, Bu', yformatter='%.0f', rot=45, legend=True)


In [128]:
# March Plot
mar_plot * mar_ewma1_plot * mar_ewma5_plot

:Overlay
   .Bars.I   :Bars   [date]   (stock_report)
   .Curve.I  :Curve   [date]   (stock_report)
   .Curve.II :Curve   [date]   (stock_report)

In [129]:
# June Plot
jun_plot * jun_ewma1_plot * jun_ewma5_plot

:Overlay
   .Bars.I   :Bars   [date]   (stock_report)
   .Curve.I  :Curve   [date]   (stock_report)
   .Curve.II :Curve   [date]   (stock_report)

In [130]:
# Septemeber Plot
sept_plot * sept_ewma1_plot * sept_ewma5_plot

:Overlay
   .Bars.I   :Bars   [date]   (stock_report)
   .Curve.I  :Curve   [date]   (stock_report)
   .Curve.II :Curve   [date]   (stock_report)

In [131]:
# December Plot
dec_plot * dec_ewma1_plot * dec_ewma5_plot

:Overlay
   .Bars.I   :Bars   [date]   (stock_report)
   .Curve.I  :Curve   [date]   (stock_report)
   .Curve.II :Curve   [date]   (stock_report)